# IMPORTADOR DE ESTACIONES DE SERVICIO Y PRECIOS POR ESTACIÓN Y PROMEDIO NACIONAL Y POR ESTADO

Actualización al 1 de septiembre de 2021

https://analyticsboutique.github.io/dashboard-simplificado/

Fuentes de datos: 

https://datos.gob.mx/busca/dataset/estaciones-de-servicio-gasolineras-y-precios-finales-de-gasolina-y-diesel

http://www.cre.gob.mx/da/PreciosPromedioMensuales.csv


In [1]:
# Dependencies
import pandas as pd
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import parse
import json
from urllib.request import urlopen
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
from datetime import date

import warnings
warnings.filterwarnings('ignore')

### NOTA IMPORTANTE: Cambiar fecha del archivo a procesar

In [2]:
# Modificar diariamente
#Fecha = '2021.08.19' 

today = date.today()

if len(str(today.month)) == 1:
    Fecha = str(today.year) + '.0' + str(today.month) + '.' + str(today.day)
else:
    Fecha = str(today.year) + '.' + str(today.month) + '.' + str(today.day)

# Promedios estatates
#Year = 2021

#Month = 7

# SECCIÓN I:

# Importador de datos de Estaciones de Servicio

In [3]:
# 
#tree = ET.parse('../Data_Raw/places_'+ Fecha +'.xml')

#
var_url = urlopen('https://bit.ly/2V1Z3sm') 
tree = ET.parse(var_url)

#
root = tree.getroot()
#root

In [4]:
# Extraemos los elementos del XML
ID = []
name = []
cre_id = []
lng = []
lat = []

for item in range (0,len(root)):
    ID.append(root[item].attrib['place_id'])
    name.append(root[item][0].text)
    cre_id.append(root[item][1].text)
    lng.append(root[item][2][0].text)
    lat.append(root[item][2][1].text)

In [5]:
# Imprime el número de registros
print(len(ID),len(name),len(cre_id),len(lat),len(lng))

12914 12914 12914 12914 12914


In [6]:
# Convertimos a DATAFRAME
Gasolinerias = pd.DataFrame({ 'ID' : ID, 'name' : name, 'cre_id' : cre_id, 'lat' : lat, 'lng' : lng })
Gasolinerias.head()

,ID,name,cre_id,lat,lng
0,2039,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",PL/658/EXP/ES/2015,32.47641,-116.9214
1,2040,"LAS MEJORES ESTACIONES, S.A DE C.V",PL/902/EXP/ES/2015,20.3037,-99.74484
2,2041,"DIAZ GAS, S.A. DE C.V.",PL/760/EXP/ES/2015,31.71947,-106.4514
3,2042,"COMBU-EXPRESS, S.A. DE C.V.",PL/825/EXP/ES/2015,20.71413,-103.3042
4,2043,"PETROMAX, S.A. DE C.V.",PL/585/EXP/ES/2015,26.03787,-98.29977


In [7]:
# Guardamos en CSV
Gasolinerias.to_csv('../Data_CSV/places_'+ Fecha +'.csv', sep = ',', encoding = 'utf-8-sig', index = False)

# Importador de datos de Precios de Gasolina en Estaciones de Servicio

In [8]:
# 
#tree = ET.parse('../Data_Raw/prices_'+ Fecha +'.xml')

#
var_url = urlopen('https://bit.ly/2JNcTha') 
tree = ET.parse(var_url)

#
root = tree.getroot()
#root

In [9]:
# Extraemos los elementos del XML
ID = []
tipo = []
gas_price = []

for item in range (0,len(root)):
    ID.append(root[item].attrib['place_id'])
    tipo.append(root[item][0].attrib['type']) 
    gas_price.append(root[item].findtext('gas_price'))

In [10]:
# Imprime el número de registros
print(len(ID),len(tipo),len(gas_price))

19555 19555 19555


In [11]:
# Convertimos a DATAFRAME
Precios = pd.DataFrame({ 'ID' : ID, 'tipo' : tipo, 'gas_price' : gas_price })
Precios.head()

,ID,tipo,gas_price
0,11703,regular,20.99
1,11703,premium,22.99
2,11702,regular,20.99
3,11701,regular,16.49
4,11700,regular,20.75


In [12]:
# Guardamos en CSV
Precios.to_csv('../Data_CSV/precios_'+ Fecha +'.csv', sep = ',', encoding = 'utf-8-sig', index = False)

# Juntamos datos de Estaciones de Servicio, Precios de Gasolina y Franquicia

In [13]:
# Selecccionamos por tipo de gasolina
Precios_regular = Precios[Precios['tipo'] == 'regular']
Precios_regular.head()

,ID,tipo,gas_price
0,11703,regular,20.99
2,11702,regular,20.99
3,11701,regular,16.49
4,11700,regular,20.75
5,11699,regular,20.17


In [14]:
# Selecccionamos por tipo de gasolina
Precios_premium = Precios[Precios['tipo'] == 'premium']
Precios_premium.head()

,ID,tipo,gas_price
1,11703,premium,22.99
6,11699,premium,22.15
14,3153,premium,22.39
19,11692,premium,22.4
23,11395,premium,22.99


In [15]:
# Selecccionamos por tipo de gasolina
Precios_diesel = Precios[Precios['tipo'] == 'diesel']
Precios_diesel.head()

,ID,tipo,gas_price
8,11698,diesel,23
15,3153,diesel,21.49
17,11692,diesel,22.45
25,11394,diesel,22.19
30,11391,diesel,20.65


In [16]:
# Juntamos datos:

# Regular
Gasolinerias_Precios = Gasolinerias

Gasolinerias_Precios = pd.merge(Gasolinerias_Precios, Precios_regular, on = 'ID', how='outer')

Gasolinerias_Precios = Gasolinerias_Precios.rename(columns={'gas_price': 'regular'})

Gasolinerias_Precios = Gasolinerias_Precios[['ID', 'name', 'cre_id', 'lat', 'lng', 'regular']]

Gasolinerias_Precios.head()

,ID,name,cre_id,lat,lng,regular
0,2039,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",PL/658/EXP/ES/2015,32.47641,-116.9214,19.99
1,2040,"LAS MEJORES ESTACIONES, S.A DE C.V",PL/902/EXP/ES/2015,20.3037,-99.74484,20.65
2,2041,"DIAZ GAS, S.A. DE C.V.",PL/760/EXP/ES/2015,31.71947,-106.4514,16.37
3,2042,"COMBU-EXPRESS, S.A. DE C.V.",PL/825/EXP/ES/2015,20.71413,-103.3042,20.99
4,2043,"PETROMAX, S.A. DE C.V.",PL/585/EXP/ES/2015,26.03787,-98.29977,16.99


In [17]:
# Premium
Gasolinerias_Precios = pd.merge(Gasolinerias_Precios, Precios_premium, on = 'ID', how='outer')

Gasolinerias_Precios = Gasolinerias_Precios.rename(columns={'gas_price': 'premium'})

Gasolinerias_Precios = Gasolinerias_Precios[['ID', 'name', 'cre_id', 'lat', 'lng', 'regular', 'premium']]

Gasolinerias_Precios.head()

,ID,name,cre_id,lat,lng,regular,premium
0,2039,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",PL/658/EXP/ES/2015,32.47641,-116.9214,19.99,22.99
1,2040,"LAS MEJORES ESTACIONES, S.A DE C.V",PL/902/EXP/ES/2015,20.3037,-99.74484,20.65,NaN
2,2041,"DIAZ GAS, S.A. DE C.V.",PL/760/EXP/ES/2015,31.71947,-106.4514,16.37,NaN
3,2042,"COMBU-EXPRESS, S.A. DE C.V.",PL/825/EXP/ES/2015,20.71413,-103.3042,20.99,NaN
4,2043,"PETROMAX, S.A. DE C.V.",PL/585/EXP/ES/2015,26.03787,-98.29977,16.99,NaN


In [18]:
# Regular
Gasolinerias_Precios = pd.merge(Gasolinerias_Precios, Precios_diesel, on = 'ID', how='outer')

Gasolinerias_Precios = Gasolinerias_Precios.rename(columns={'gas_price': 'diesel'})

Gasolinerias_Precios = Gasolinerias_Precios[['ID', 'name', 'cre_id', 'lat', 'lng', 'regular', 'premium', 'diesel']]

Gasolinerias_Precios.head()

,ID,name,cre_id,lat,lng,regular,premium,diesel
0,2039,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",PL/658/EXP/ES/2015,32.47641,-116.9214,19.99,22.99,21.19
1,2040,"LAS MEJORES ESTACIONES, S.A DE C.V",PL/902/EXP/ES/2015,20.3037,-99.74484,20.65,NaN,NaN
2,2041,"DIAZ GAS, S.A. DE C.V.",PL/760/EXP/ES/2015,31.71947,-106.4514,16.37,NaN,NaN
3,2042,"COMBU-EXPRESS, S.A. DE C.V.",PL/825/EXP/ES/2015,20.71413,-103.3042,20.99,NaN,NaN
4,2043,"PETROMAX, S.A. DE C.V.",PL/585/EXP/ES/2015,26.03787,-98.29977,16.99,NaN,NaN


In [19]:
# Is NA's y revisar

Gasolinerias_Precios[Gasolinerias_Precios['cre_id'].isnull()]

,ID,name,cre_id,lat,lng,regular,premium,diesel


In [20]:
# Lectura de la base de franquicias:
Gasolinerias_Franquicias = pd.read_csv('../Data_CSV/Base_Franquicias_2020.10.08.csv')

Gasolinerias_Franquicias = Gasolinerias_Franquicias.rename(columns={'ID_CRE': 'ID'})

Gasolinerias_Franquicias.head()

,ID,Franquicia_Marca
0,2039,PEMEX
1,2040,PERC
2,2041,PEMEX
3,2042,Combu-Express
4,2043,Petro Seven


In [21]:
# Convertimos ID en número
Gasolinerias_Precios['ID'] = pd.to_numeric(Gasolinerias_Precios['ID'])

Gasolinerias_Precios.dtypes

ID          int64
name       object
cre_id     object
lat        object
lng        object
regular    object
premium    object
diesel     object
dtype: object

In [22]:
# Verificamos el tipo de dato del ID
Gasolinerias_Franquicias.dtypes

ID                   int64
Franquicia_Marca    object
dtype: object

In [23]:
# Unión con la base de precios:
Gasolinerias_Precios = pd.merge(Gasolinerias_Precios, Gasolinerias_Franquicias, on = 'ID', how='outer')

Gasolinerias_Precios.head()

,ID,name,cre_id,lat,lng,regular,premium,diesel,Franquicia_Marca
0,2039,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",PL/658/EXP/ES/2015,32.47641,-116.9214,19.99,22.99,21.19,PEMEX
1,2040,"LAS MEJORES ESTACIONES, S.A DE C.V",PL/902/EXP/ES/2015,20.3037,-99.74484,20.65,NaN,NaN,PERC
2,2041,"DIAZ GAS, S.A. DE C.V.",PL/760/EXP/ES/2015,31.71947,-106.4514,16.37,NaN,NaN,PEMEX
3,2042,"COMBU-EXPRESS, S.A. DE C.V.",PL/825/EXP/ES/2015,20.71413,-103.3042,20.99,NaN,NaN,Combu-Express
4,2043,"PETROMAX, S.A. DE C.V.",PL/585/EXP/ES/2015,26.03787,-98.29977,16.99,NaN,NaN,Petro Seven


In [24]:
# Is NA's y revisar

Gasolinerias_Precios[Gasolinerias_Precios['ID'].isnull()]

,ID,name,cre_id,lat,lng,regular,premium,diesel,Franquicia_Marca


In [25]:
Gasolinerias_Precios.head()

,ID,name,cre_id,lat,lng,regular,premium,diesel,Franquicia_Marca
0,2039,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",PL/658/EXP/ES/2015,32.47641,-116.9214,19.99,22.99,21.19,PEMEX
1,2040,"LAS MEJORES ESTACIONES, S.A DE C.V",PL/902/EXP/ES/2015,20.3037,-99.74484,20.65,NaN,NaN,PERC
2,2041,"DIAZ GAS, S.A. DE C.V.",PL/760/EXP/ES/2015,31.71947,-106.4514,16.37,NaN,NaN,PEMEX
3,2042,"COMBU-EXPRESS, S.A. DE C.V.",PL/825/EXP/ES/2015,20.71413,-103.3042,20.99,NaN,NaN,Combu-Express
4,2043,"PETROMAX, S.A. DE C.V.",PL/585/EXP/ES/2015,26.03787,-98.29977,16.99,NaN,NaN,Petro Seven


In [26]:
# Guardamos en CSV
Gasolinerias_Precios.to_csv('../Data_CSV/places_&_prices_'+ Fecha +'.csv', sep = ',', encoding = 'utf-8-sig', 
                            index = False)

In [27]:
# Guardamos en CSV
Gasolinerias_Precios.to_json('Data/Price_Stations.json', orient='table', date_format = 'iso', index = False)

# SECCIÓN II:

# Importador de datos de Estaciones de Servicio

### NOTA: Modificar el nombre del archivo para actualizar datos (1 vez cada mes)

http://transparenciacre.westcentralus.cloudapp.azure.com/PNT/73/III/E/PL/Precios_promedio_diarios_y_mensuales_en_estaciones_de_servicio.xlsx

In [28]:
# Lectura de la base de franquicias:
#PreciosPromedioMensuales = pd.read_csv('../Data_Raw/PreciosPromedioMensuales_2021.08.14.csv', encoding = 'latin-1', 
#                              skiprows = 1)

PreciosPromedioMensuales = pd.read_csv('http://www.cre.gob.mx/da/PreciosPromedioMensuales.csv', encoding = 'latin-1', 
                              skiprows = 1)

PreciosPromedioMensuales.tail()

,Entidad federativa,Gasolina mínimo 87 octanos,Año reporte,Mes,Unnamed: 4,Entidad federativa.1,Gasolina mínimo 91 octanos,Año reporte.1,Mes.1,Unnamed: 9,Entidad federativa.2,Diésel,Año reporte.2,Mes.2,Unnamed: 14,Unnamed: 15,FechaCalendario,Gasolina mínimo 87 octanos.1,Gasolina mínimo 91 octanos.1,Diésel.1
1915,Zacatecas,20.86,2021,8,NaN,Zacatecas,22.59,2021,8,NaN,Zacatecas,21.86,2021,8,NaN,NaN,NaN,NaN,NaN,NaN
1916,Zacatecas,20.77,2021,9,NaN,Zacatecas,22.59,2021,9,NaN,Zacatecas,21.72,2021,9,NaN,NaN,NaN,NaN,NaN,NaN
1917,Zacatecas,20.72,2021,10,NaN,Zacatecas,22.54,2021,10,NaN,Zacatecas,21.59,2021,10,NaN,NaN,NaN,NaN,NaN,NaN
1918,Zacatecas,20.60,2021,11,NaN,Zacatecas,22.51,2021,11,NaN,Zacatecas,21.53,2021,11,NaN,NaN,NaN,NaN,NaN,NaN
1919,Zacatecas,20.71,2021,12,NaN,Zacatecas,22.60,2021,12,NaN,Zacatecas,21.65,2021,12,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# Fechas Promedios estatates

Year = PreciosPromedioMensuales['Año reporte'][PreciosPromedioMensuales.shape[0]-1]

Month = PreciosPromedioMensuales['Mes'][PreciosPromedioMensuales.shape[0]-1]

In [30]:
# Seleccionamos datos de precios promedio por Estado
Estados_Precios = PreciosPromedioMensuales[['Entidad federativa', 'Año reporte', 'Mes',
                                            'Gasolina mínimo 87 octanos',
                                            'Gasolina mínimo 91 octanos',
                                            'Diésel']]

Estados_Precios = Estados_Precios.rename(columns={ 'Entidad federativa':'Entidad', 'Año reporte':'Year', 
                                                  'Mes':'Month', 'Gasolina mínimo 87 octanos':'Gas87',
                                                  'Gasolina mínimo 91 octanos':'Gas91', 'Diésel':'Diesel' })

Estados_Precios.head()

,Entidad,Year,Month,Gas87,Gas91,Diesel
0,Aguascalientes,2017,1,16.19,17.96,17.28
1,Aguascalientes,2017,2,16.22,17.99,17.31
2,Aguascalientes,2017,3,16.10,17.86,17.16
3,Aguascalientes,2017,4,16.12,17.85,17.09
4,Aguascalientes,2017,5,16.03,17.78,16.97


In [31]:
#
Estados_Precios.tail()

,Entidad,Year,Month,Gas87,Gas91,Diesel
1915,Zacatecas,2021,8,20.86,22.59,21.86
1916,Zacatecas,2021,9,20.77,22.59,21.72
1917,Zacatecas,2021,10,20.72,22.54,21.59
1918,Zacatecas,2021,11,20.60,22.51,21.53
1919,Zacatecas,2021,12,20.71,22.60,21.65


In [32]:
# Agregamos coordenadas:
Estados_Precios['lng'] = 0
Estados_Precios['lat'] = 0

Estados_Precios.loc[Estados_Precios['Entidad'] == 'Aguascalientes', 'lng'] = -102.291
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Aguascalientes', 'lat'] = 21.8818
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Baja California', 'lng'] = -115.283
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Baja California', 'lat'] = 30.8406
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Baja California Sur', 'lng'] = -111.666
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Baja California Sur', 'lat'] = 26.0444
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Campeche', 'lng'] = -90.5349
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Campeche', 'lat'] = 19.8301
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Chiapas', 'lng'] = -93.1292
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Chiapas', 'lat'] = 16.7569
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Chihuahua', 'lng'] = -106.0691
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Chihuahua', 'lat'] = 28.633
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Ciudad de México', 'lng'] = -99.1332
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Ciudad de México', 'lat'] = 19.4326
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Coahuila de Zaragoza', 'lng'] = -101.7068
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Coahuila de Zaragoza', 'lat'] = 27.0587
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Colima', 'lng'] = -103.7241
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Colima', 'lat'] = 19.2452
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Durango', 'lng'] = -104.6532
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Durango', 'lat'] = 24.0277
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Guanajuato', 'lng'] = -101.2574
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Guanajuato', 'lat'] = 21.019
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Guerrero', 'lng'] = -99.5451
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Guerrero', 'lat'] = 99.5451
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Hidalgo', 'lng'] = -98.7624
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Hidalgo', 'lat'] = 20.0911
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Jalisco', 'lng'] = -103.3494
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Jalisco', 'lat'] = 20.6595
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Michoacán de Ocampo', 'lng'] = -101.7068
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Michoacán de Ocampo', 'lat'] = 19.5665
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Morelos', 'lng'] = -99.1013
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Morelos', 'lat'] = 18.6813
Estados_Precios.loc[Estados_Precios['Entidad'] == 'México', 'lng'] = -99.7233
Estados_Precios.loc[Estados_Precios['Entidad'] == 'México', 'lat'] = 19.4969
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Nayarit', 'lng'] = -104.8455
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Nayarit', 'lat'] = 21.7514
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Nuevo León', 'lng'] = -99.9962
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Nuevo León', 'lat'] = 25.5922
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Oaxaca', 'lng'] = -96.7266
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Oaxaca', 'lat'] = 17.0732
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Puebla', 'lng'] = -98.20193
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Puebla', 'lat'] = 19.04334
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Querétaro', 'lng'] = -100.38806
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Querétaro', 'lat'] = 20.58806
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Quintana Roo', 'lng'] = -88.4791
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Quintana Roo', 'lat'] = 19.1817
Estados_Precios.loc[Estados_Precios['Entidad'] == 'San Luis Potosí', 'lng'] = -100.97916
Estados_Precios.loc[Estados_Precios['Entidad'] == 'San Luis Potosí', 'lat'] = 22.14982
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Sinaloa', 'lng'] = -107.4795
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Sinaloa', 'lat'] = 25.1721
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Sonora', 'lng'] = -110.66667
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Sonora', 'lat'] = 29.33333
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Tabasco', 'lng'] = -92.6189
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Tabasco', 'lat'] = 17.8409
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Tamaulipas', 'lng'] = -98.8363
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Tamaulipas', 'lat'] = 24.2669
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Tlaxcala', 'lng'] = -98.19982
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Tlaxcala', 'lat'] = 19.31905
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Veracruz de Ignacio de la Llave', 'lng'] = -96.1428986
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Veracruz de Ignacio de la Llave', 'lat'] = 19.1809502
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Yucatán', 'lng'] = -89.05037
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Yucatán', 'lat'] = 19.69302
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Zacatecas', 'lng'] = -102.5832
Estados_Precios.loc[Estados_Precios['Entidad'] == 'Zacatecas', 'lat'] = 22.7709

In [33]:
# Guardamos en CSV
Estados_Precios.to_csv('../Data_CSV/Price_State_ALL.csv', sep = ',', encoding = 'utf-8-sig', index = False)

In [34]:
#
# NOTA: ADECUAR SELECCIÓN DE MES AL INICIO DEL NOTEBOOK
#
Price_State = Estados_Precios.loc[(Estados_Precios['Year'] == Year) & (Estados_Precios['Month'] == Month)]

#
Price_State['latitude'] = Price_State['lat'].astype(float)
Price_State['longitude'] = Price_State['lng'].astype(float)

#
useful_cols = ['Entidad', 'Year', 'Month', 'Gas87', 'Gas91', 'Diesel', 'latitude', 'longitude']
price_state_2 = Price_State[useful_cols]

price_state_2.head()

,Entidad,Year,Month,Gas87,Gas91,Diesel,latitude,longitude
59,Aguascalientes,2021,12,20.25,22.69,21.62,21.8818,-102.2910
119,Baja California,2021,12,19.37,21.53,20.80,30.8406,-115.2830
179,Baja California Sur,2021,12,20.84,22.75,22.25,26.0444,-111.6660
239,Campeche,2021,12,21.69,23.11,23.08,19.8301,-90.5349
299,Chiapas,2021,12,20.71,22.27,22.32,16.7569,-93.1292


In [35]:
# Guardamos en CSV
price_state_2.to_json('Data/Price_State.json', orient='table', date_format = 'iso', index = False)

In [36]:
# Function that TRANSFORM dataframe to GEOJSON:
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    """
    Turn a dataframe containing point data into a geojson formatted python dictionary
    
    df : the dataframe to convert to geojson
    properties : a list of columns in the dataframe to turn into geojson feature properties
    lat : the name of the column in the dataframe that contains latitude data
    lon : the name of the column in the dataframe that contains longitude data
    """
    
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}
    
    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        
        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        
        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson
    
# Return GEOJSON:
useful_columns = ['Entidad', 'Year', 'Month', 'Gas87', 'Gas91', 'Diesel']
geojson_dict = df_to_geojson(price_state_2, properties = useful_columns)

#return geojson_dict

In [37]:
# Guardamos en CSV
with open('Data/Price_State.geojson', 'w', encoding = 'utf-8') as f:
    json.dump(geojson_dict, f, ensure_ascii = False)

In [38]:
# Seleccionamos datos de la serie de precios promedio diario
Serie_Precios = PreciosPromedioMensuales[['FechaCalendario', 'Gasolina mínimo 87 octanos.1', 
                                          'Gasolina mínimo 91 octanos.1', 'Diésel.1']]

Serie_Precios = Serie_Precios.rename(columns={ 'FechaCalendario':'Fecha', 'Gasolina mínimo 87 octanos.1':'Gas87',
                                                  'Gasolina mínimo 91 octanos.1':'Gas91', 'Diésel.1':'Diesel' })

Serie_Precios = Serie_Precios.dropna()

Serie_Precios.head()

,Fecha,Gas87,Gas91,Diesel
0,01/01/2017,15.99,17.79,17.05
1,02/01/2017,16.02,17.82,17.08
2,03/01/2017,16.02,17.83,17.08
3,04/01/2017,16.02,17.83,17.08
4,05/01/2017,16.02,17.83,17.08


In [39]:
Serie_Precios.tail()

,Fecha,Gas87,Gas91,Diesel
1821,27/12/2021,20.40,22.55,21.92
1822,28/12/2021,20.45,22.59,21.96
1823,29/12/2021,20.49,22.59,21.99
1824,30/12/2021,20.51,22.61,22.02
1825,31/12/2021,20.53,22.62,22.04


In [40]:
# Guardamos en CSV
Serie_Precios.to_csv('../Data_CSV/Price_Time_Serie.csv', sep = ',', encoding = 'utf-8-sig', index = False)

In [41]:
# Guardamos en JSON
Serie_Precios.to_json('Data/Price_Time_Serie.json', orient = 'table', date_format = 'iso', index = False)